In [1]:
import os
import dotenv
from pathlib import Path

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import (
    WebBaseLoader, 
    PyPDFLoader, 
    Docx2txtLoader,
)
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

os.environ["USER_AGENT"] = "MyFastAPIApp/1.0"
dotenv.load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
# Load docs

doc_paths = [
    "docs/Abhijith-Kotak Resume-AI Engineer role.pdf"
]

docs = [] 
for doc_file in doc_paths:
    file_path = Path(doc_file)

    try:
        if doc_file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif doc_file.endswith(".docx"):
            loader = Docx2txtLoader(file_path)
        elif doc_file.endswith(".txt") or doc_file.name.endswith(".md"):
            loader = TextLoader(file_path)
        else:
            print(f"Document type {doc_file.type} not supported.")
            continue

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {doc_file.name}: {e}")


# Load URLs

url = "https://docs.streamlit.io/develop/quick-reference/release-notes"
try:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())

except Exception as e:
    print(f"Error loading document from {url}: {e}")

In [3]:
docs

[Document(metadata={'source': 'docs\\Abhijith-Kotak Resume-AI Engineer role.pdf', 'page': 0}, page_content='Abhijith  K V – Data Scientist  Resume  \nabhijithkv56@gmail.com  | 9321670581  |GitHub: https://github.com/abhijithkv056  \n \nACADEMIC PROFILE  \n \nIIT Madras ,                     Bachelor of Technology (B. Tech) , CGPA  7.58 /10  \nIJPS School, Kerala                   Class XII – Computer Science , CGPA 9.4 / 10  \n \nWORK EXPERI ENCE \n \nData Scientist                Nine Stacks Gaming LLP (Internship)    Nov 2020 – Feb 2021 \n \n• Experienced in Build, train and deploy Machine Learning Models  and working in production  environment setup.  \n• Conceptualized  a machine learning Forecasting model  using XG Boost  to predict the customer  lifetime  \nvalue for helping to improving customer retention taking decisions on customized gamification.  \n• Conceptualized a flag system  to improve the identify early dropout user  saving ~ 15% reward points spend  \n• Proficient in 

In [5]:
# Split docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

document_chunks = text_splitter.split_documents(docs)
document_chunks

[Document(metadata={'source': 'docs\\Abhijith-Kotak Resume-AI Engineer role.pdf', 'page': 0}, page_content='Abhijith  K V – Data Scientist  Resume  \nabhijithkv56@gmail.com  | 9321670581  |GitHub: https://github.com/abhijithkv056  \n \nACADEMIC PROFILE  \n \nIIT Madras ,                     Bachelor of Technology (B. Tech) , CGPA  7.58 /10  \nIJPS School, Kerala                   Class XII – Computer Science , CGPA 9.4 / 10  \n \nWORK EXPERI ENCE \n \nData Scientist                Nine Stacks Gaming LLP (Internship)    Nov 2020 – Feb 2021 \n \n• Experienced in Build, train and deploy Machine Learning Models  and working in production  environment setup.  \n• Conceptualized  a machine learning Forecasting model  using XG Boost  to predict the customer  lifetime  \nvalue for helping to improving customer retention taking decisions on customized gamification.  \n• Conceptualized a flag system  to improve the identify early dropout user  saving ~ 15% reward points spend  \n• Proficient in 

In [16]:
len(document_chunks[1].page_content)

938

In [17]:
# Tokenize and load the documents to the vector store

vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=OpenAIEmbeddings(),
)

In [18]:
vector_db

In [33]:
# Retrieve

def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the option/input chosen by the user, mention if the option is correct or not. In both cases, give a short explanation focussing on most recent question."),
    ])
    #log_retriever = retriever.invoke(prompt)

    #for logs in log_retriever:

        #print(f"Content of log_retriver: {logs.page_content}")
        #print(f"Metadata of log_retriver: {logs.metadata}")

    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [34]:
def get_conversational_rag_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are a quiz master that ask questions to user. you will ask user a question and give 4 options. only one opion will be correct.
        You will have some context to help with your asking the questions and deciding the correct option, but now always would be completely related or helpful.
        You can also use your knowledge to assist answering the user's queries.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    print(type(stuff_documents_chain), " its type")

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [38]:
# Augmented Generation

llm_stream_openai = ChatOpenAI(
    model="gpt-4o",  # Here you could use "o1-preview" or "o1-mini" if you already have access to them
    temperature=0.3,
    streaming=True,
)

llm_stream_anthropic = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0.3,
    streaming=True,
)

llm_stream = llm_stream_openai  # Select between OpenAI and Anthropic models for the response

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "quiz me"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]
print(messages)

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

[HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi there! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='quiz me', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.runnables.base.RunnableBinding'>  its type
Sure! Here's a question for you:

Which of the following is a cloud service provided by Amazon Web Services (AWS)?

A) Azure DevOps  
B) Google Cloud Storage  
C) Amazon S3  
D) IBM Cloud Functions  

What's your answer?